<a href="https://colab.research.google.com/github/gutruaircraft/google-colab/blob/master/VGG_19_With_predection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Rose(1).jpg to Rose(1).jpg


In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['Flowers_small.zip']), 'r')
data.extractall()

In [0]:
!ls

sample_data


**VGG - 19**

In [0]:
# USAGE
# python train.py --dataset dataset --model pokedex.model --labelbin lb.pickle

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#from smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

# construct the argument parse and parse the arguments

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 30
INIT_LR = 1e-4  #intial learning rate
BS = 64
IMAGE_DIMS = (224,224, 3)

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images("Flowers_small")))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
#print (labels)
#print (data)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.30)

# construct the image generator for data augmentation
'''aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")'''



# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import regularizers
from keras import backend as K
from keras import initializers

class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1


		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3),kernel_initializer="he_normal", padding="same",input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal", padding="same"))   #https://keras.io/initializers/
		model.add(Activation("relu"))
		#model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.5))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal", padding="same"))  #TAKE different intializers from above link
		model.add(Activation("relu"))
		model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		#model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.20))
    
		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(256, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(256, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(256, (3,3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))  
		model.add(Conv2D(256, (3,3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu")) 
		#model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.20))

		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		#model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.20))    
    
		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		model.add(Conv2D(512, (3, 3), kernel_initializer="he_normal", padding="same"))
		model.add(Activation("relu"))
		#model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		#model.add(Dropout(0.20))
    
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(4096))  # add seperately after training conv layers   kernel_initializer="he_normal"
		model.add(Activation("relu"))
		
              
		model.add(Dense(4096))
		model.add(Activation("relu"))
		#model.add(BatchNormalization())
		#model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		print (model.summary())
		# return the constructed network architecture 9840298559
		return model



# initialize the model
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)                 #https://keras.io/optimizers/
model.compile(loss="categorical_crossentropy", optimizer=opt,       #https://keras.io/losses/
	metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(
	trainX, trainY, batch_size=BS,
	validation_data=(testX, testY),
	epochs=EPOCHS)

# save the model to disk
print("[INFO] serializing network...")
model.save("VGG_19.model")
model.save('model.h5')


# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open("VGG_19.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

#predict the model
#y_model = model.predict(testX)
#print(y_model)
#Accuracy score
#accuracy=accuracy_score(testY, y_model)
#print("Accuracy: %.2f%%" % (accuracy * 100.0)) 

#printing score
#score = model.score(y_model)
#print(score)

loss_and_metrics = model.evaluate(testX, testY)
print(loss_and_metrics)

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy vgg 19")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("VGG_19.png")
train_acc = np.mean(H.history["acc"])
val_acc = np.mean(H.history["val_acc"])
print("Average_training_accuracy =")
print(train_acc)
print("Average_validation_accuracy = ")
print(val_acc)

[INFO] loading images...
[INFO] data matrix: 1176.00MB
[INFO] compiling model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_____________________________________________________________

In [0]:
from google.colab import files
uploaded = files.download()

In [0]:

import cv2
# Modify 'test1.jpg' and 'test2.jpg' to the images you want to predict on

from keras.models import load_model
from keras.preprocessing import image
import numpy as np
# dimensions of our images
img_width, img_height = 224, 224
#converting to greyscale
img3 = cv2.imread('Rose(1).jpg')
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
dim = (img_width, img_height)
# resize image
img3 = cv2.resize(img3, dim, interpolation = cv2.INTER_AREA) 
# load the model we saved
model = load_model('model.h5')
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# predicting images
#img = image.load_img('test1.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img3)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict_classes(images, batch_size=10)
print (classes)






Using TensorFlow backend.
W0620 16:15:36.924465 139974612219776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 16:15:36.960462 139974612219776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0620 16:15:36.999905 139974612219776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0620 16:15:37.195111 139974612219776 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0620 16:15:40.532422 1399746122

ValueError: ignored